<a href="https://colab.research.google.com/github/ARPITA1312/DEEP_LEARNING/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Create CNN Model and optimize it using Keras Tuner

In [7]:
!pip install keras-tuner

In [8]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [9]:
print(tf.__version__)

2.15.0


In [10]:
fashion_mnist=keras.datasets.fashion_mnist

In [11]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [12]:
train_images=train_images/255.0
test_images=test_images/255.0

In [13]:
train_images[0].shape

(28, 28)

In [14]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [15]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

  return model

In [17]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [18]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 3 Complete [00h 07m 24s]
val_accuracy: 0.9056666493415833

Best val_accuracy So Far: 0.9056666493415833
Total elapsed time: 00h 36m 09s

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
112               |32                |conv_1_filter
3                 |5                 |conv_1_kernel
64                |48                |conv_2_filter
5                 |3                 |conv_2_kernel
80                |96                |dense_1_units
0.01              |0.001             |learning_rate

Epoch 1/3
1688/1688 [==============================] - 814s 482ms/step - loss: 0.5356 - accuracy: 0.8239 - val_loss: 0.4734 - val_accuracy: 0.8290
Epoch 2/3
1494/1688 [=========================>....] - ETA: 1:27 - loss: 0.3726 - accuracy: 0.8612

In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

In [ ]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)